In [3]:
import numpy as np
import sys
from mpi4py import MPI
import PySTKFMM
import timer
import matplotlib.pyplot as plt

In [5]:
# FMM parameters
mult_order = 10
max_pts = 128
pbc = PySTKFMM.PAXIS.NONE
kernel = PySTKFMM.KERNEL.Stokes

### Single stokeslet near a no-slip wall (Oseen problem)

- Q: where is the wall?
- Simulation is getting stuck?!

In [ ]:
# Create sources and targets coordinates
ntrg = 2

src_coord = np.array([[1.5, 1.5, 1.5]]) # be careful not to set it with integer array
nsrc = len(src_coord)
                         
trg_coord = np.array([src_coord[0] for i in range(ntrg)])
for i in range(ntrg):
    trg_coord[i, 0] += i * 0.04

myFMM = PySTKFMM.StkWallFMM(mult_order, max_pts, pbc, kernel)
myFMM.showActiveKernels()
kdimSL, kdimDL, kdimTrg = myFMM.getKernelDimension(kernel)

# Create sources and target values
src_value = np.array([[1.,0,0]]) # unit force pointing in x-direction
trg_value = np.zeros((ntrg, kdimTrg)) # target values = velocities

# Set tree
myFMM.setBox([-2.0, -2.0, -2.0], 4)
myFMM.setPoints(nsrc, src_coord, ntrg, trg_coord, 0, None)
myFMM.setupTree(kernel)

# Evaluate FMM
myFMM.evaluateFMM(kernel, nsrc, src_value, ntrg, trg_value, 0, None)

In [ ]:
trg_abs = np.linalg.norm(trg_value, axis=1)

xs = trg_coord[:,0]
ys = trg_abs

xs_slope = xs[1:]
ys_slope = ys[1] * xs[1] * xs[1:] ** -1 # ~ 1/r

plt.plot(xs, ys, 'o')
plt.plot(xs_slope, ys_slope,'--')
#plt.yscale('log')
#plt.xscale('log')